In [47]:
import pandas as pd
import json
import psycopg2
import logging
import os

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [48]:
def read_profiles_from_file(file_path):
    profiles = []
    with open(file_path, 'r') as file:
        for line in file:
            # Remove any leading,trailing whitespace & parse file
            profile_data = json.loads(line.strip())
            profiles.append(profile_data)
    return profiles


# Reading in the profiles as individual lines from the text file

In [49]:
person_file_path = 'data/us_person_profile.txt'
company_file_path = 'data/us_person_profile.txt'
person_profiles = read_profiles_from_file(person_file_path)
company_profiles = read_profiles_from_file(company_file_path)


# An example of the first profile

## Person

In [50]:
person_profiles[0]

{'public_identifier': 'none',
 'profile_pic_url': 'https://static-exp1.licdn.com/sc/h/244xhbkr7g40x6bsu4gi6q4ry',
 'background_cover_image_url': 'https://static-exp1.licdn.com/sc/h/5q92mjc5c51bjlwaj3rs9aa82',
 'first_name': 'Jeff',
 'last_name': 'B.',
 'full_name': 'Jeff B.',
 'occupation': 'AVP, Life Underwriting at Illinois Mutual',
 'headline': 'Illinois Mutual',
 'summary': None,
 'country': 'US',
 'country_full_name': 'United States of America',
 'city': 'Normal',
 'state': 'Illinois',
 'experiences': [{'starts_at': {'day': 1, 'month': 8, 'year': 2018},
   'ends_at': None,
   'company': 'Illinois Mutual',
   'company_linkedin_profile_url': 'https://www.linkedin.com/company/illinois-mutual',
   'title': 'AVP, Life Underwriting',
   'description': None,
   'location': 'Peoria, Illinois Area',
   'logo_url': 'https://media-exp1.licdn.com/dms/image/C4E0BAQEbxRTeOThzQg/company-logo_100_100/0/1523283330240?e=1655942400&v=beta&t=LY8dpbCLm80UJviuyTy6qyEvTNX6WetT-VGkEWijjXc'},
  {'starts_a

## Company

In [51]:

print(company_profiles[0])

{'public_identifier': 'none', 'profile_pic_url': 'https://static-exp1.licdn.com/sc/h/244xhbkr7g40x6bsu4gi6q4ry', 'background_cover_image_url': 'https://static-exp1.licdn.com/sc/h/5q92mjc5c51bjlwaj3rs9aa82', 'first_name': 'Jeff', 'last_name': 'B.', 'full_name': 'Jeff B.', 'occupation': 'AVP, Life Underwriting at Illinois Mutual', 'headline': 'Illinois Mutual', 'summary': None, 'country': 'US', 'country_full_name': 'United States of America', 'city': 'Normal', 'state': 'Illinois', 'experiences': [{'starts_at': {'day': 1, 'month': 8, 'year': 2018}, 'ends_at': None, 'company': 'Illinois Mutual', 'company_linkedin_profile_url': 'https://www.linkedin.com/company/illinois-mutual', 'title': 'AVP, Life Underwriting', 'description': None, 'location': 'Peoria, Illinois Area', 'logo_url': 'https://media-exp1.licdn.com/dms/image/C4E0BAQEbxRTeOThzQg/company-logo_100_100/0/1523283330240?e=1655942400&v=beta&t=LY8dpbCLm80UJviuyTy6qyEvTNX6WetT-VGkEWijjXc'}, {'starts_at': {'day': 1, 'month': 1, 'year': 2

# Connect to the Gcloud SQL instance and insert into table

In [52]:

def create_company_profile_table_if_not_exists(cursor):
    create_table_query = """
    CREATE TABLE IF NOT EXISTS company_profiles (
        id SERIAL PRIMARY KEY,
        linkedin_internal_id VARCHAR,
        description TEXT,
        website VARCHAR,
        industry VARCHAR,
        company_size_start INT,
        company_size_end INT,
        company_size_on_linkedin INT,
        hq_country VARCHAR,
        hq_city VARCHAR,
        hq_postal_code VARCHAR,
        hq_line_1 VARCHAR,
        hq_is_hq BOOLEAN,
        hq_state VARCHAR,
        company_type VARCHAR,
        founded_year INT,
        specialities TEXT[],
        name VARCHAR,
        tagline TEXT,
        universal_name_id VARCHAR,
        profile_pic_url VARCHAR,
        background_cover_image_url VARCHAR,
        search_id VARCHAR,
        follower_count INT
    );
    """
    cursor.execute(create_table_query)


In [55]:
def insert_company_profiles_to_postgres(profiles, connection_params):
    try:
        conn = psycopg2.connect(**connection_params)
        cursor = conn.cursor()
        
        create_company_profile_table_if_not_exists(cursor)
        
        for profile in profiles:
            insert_query = """
            INSERT INTO company_profiles (linkedin_internal_id, description, website, industry, company_size_start, company_size_end, company_size_on_linkedin, hq_country, hq_city, hq_postal_code, hq_line_1, hq_is_hq, hq_state, company_type, founded_year, specialities, name, tagline, universal_name_id, profile_pic_url, background_cover_image_url, search_id, follower_count)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            RETURNING id;
            """
            
            linkedin_internal_id = profile.get('linkedin_internal_id')
            description = profile.get('description')
            website = profile.get('website')
            industry = profile.get('industry')
            company_size = profile.get('company_size', [None, None])
            company_size_start = company_size[0]
            company_size_end = company_size[1]
            company_size_on_linkedin = profile.get('company_size_on_linkedin')
            hq = profile.get('hq', {})
            hq_country = hq.get('country')
            hq_city = hq.get('city')
            hq_postal_code = hq.get('postal_code')
            hq_line_1 = hq.get('line_1')
            hq_is_hq = hq.get('is_hq')
            hq_state = hq.get('state')
            company_type = profile.get('company_type')
            founded_year = profile.get('founded_year')
            specialities = profile.get('specialities', [])
            name = profile.get('name')
            tagline = profile.get('tagline')
            universal_name_id = profile.get('universal_name_id')
            profile_pic_url = profile.get('profile_pic_url')
            background_cover_image_url = profile.get('background_cover_image_url')
            search_id = profile.get('search_id')
            follower_count = profile.get('follower_count')
            
            data_tuple = (
                linkedin_internal_id, description, website, industry, company_size_start, company_size_end, company_size_on_linkedin,
                hq_country, hq_city, hq_postal_code, hq_line_1, hq_is_hq, hq_state, company_type, founded_year, specialities,
                name, tagline, universal_name_id, profile_pic_url, background_cover_image_url, search_id, follower_count
            )
            
            cursor.execute(insert_query, data_tuple)
            id = cursor.fetchone()[0]
            logger.info(f"Inserted company profile with ID: {id}")
        
        conn.commit()
        cursor.close()
        conn.close()
    except Exception as e:
        logger.error("Error connecting to PostgreSQL database: %s", e)
        raise

In [56]:
password = open("secrets/password.pem", "r").read().strip()
connection_params = {
        "sslmode": "verify-ca",
        "sslrootcert": "secrets/server-ca.pem",
        "sslcert": "secrets/client-cert.pem",
        "sslkey": "secrets/client-key.pem",
        "hostaddr": "34.30.107.254",
        "port": "5432",
        "user": "postgres",
        "dbname": "postgres",
        "user": "postgres",
        "password" : password
    }
insert_company_profiles_to_postgres(company_profiles, connection_params)